# Train YOLO Model with Custom Data and Export to NCNN
This notebook demonstrates how to train a YOLO model with your own images and labels, export the trained model to NCNN format, and test the exported model on a custom image for object detection.

## 0. How to Get Open Source Images and Label Them for Custom Training

### 1. Downloading Open Source Images
- Use public datasets from sources like [Roboflow Universe](https://universe.roboflow.com/), [Kaggle Datasets](https://www.kaggle.com/datasets), [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html), or [Google Dataset Search](https://datasetsearch.research.google.com/).
- For manufacturer labels, search for datasets with product images, packaging, or industrial scenes. If no dataset exists, you can use Google Images or Bing Images to download images (ensure you have the right to use them).
- Tools like [OpenCV's image downloader](https://github.com/hardikvasa/google-images-download) or [icrawler](https://github.com/hellock/icrawler) can automate image collection.

### 2. Labeling Images
- Use annotation tools such as [LabelImg](https://github.com/tzutalin/labelImg), [Roboflow Annotate](https://roboflow.com/annotate), or [CVAT](https://github.com/opencv/cvat) to draw bounding boxes around manufacturer labels or items of interest.
- Save annotations in YOLO format (one `.txt` file per image, with class and bounding box coordinates).

### 3. Creating a Custom Dataset
- Organize your dataset as described in Section 2 of this notebook: separate `images/train`, `images/val`, `labels/train`, and `labels/val` folders, and create a `data.yaml` file listing class names.

**Tip:** Roboflow can help you convert and export datasets directly in YOLO format, ready for training.

## 1. Install Required Libraries
Install the necessary libraries for YOLO training and NCNN export.

In [ ]:
# Install YOLOv5, PyTorch, and NCNN export dependencies
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install opencv-python
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
# Install onnx and onnxsim for export
!pip install onnx onnxsim
# Install ncnn2mem for NCNN conversion (requires build from source or prebuilt binary)
# See: https://github.com/nihui/ncnn2mem

## 2. Prepare Custom Dataset
Organize your images and labels in YOLO format and create a data.yaml file describing your dataset.

Your dataset directory should look like this:

```
custom_dataset/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── data.yaml
```

Example `data.yaml`:
```
train: /content/custom_dataset/images/train
val: /content/custom_dataset/images/val
nc: 2  # number of classes
names: ['class1', 'class2']
```

## 3. Train YOLOv5 Model with Custom Data
Train the YOLOv5 model using your custom dataset.

In [ ]:
# Train YOLOv5 model with custom data
# Update the path to your data.yaml file
custom_data_yaml = '/content/custom_dataset/data.yaml'  # Update this path
output_dir = 'runs/train/custom_yolo_model'   # Directory to save trained model

!python train.py --img 640 --batch 16 --epochs 50 --data $custom_data_yaml --weights yolov5s.pt --project runs/train --name custom_yolo_model

# The best model will be saved in output_dir/weights/best.pt

## 4. Export Trained Model to NCNN Format
Export the trained YOLOv5 model to ONNX, then convert ONNX to NCNN format.

In [ ]:
# Export trained YOLOv5 model to ONNX
best_weights = 'runs/train/custom_yolo_model/weights/best.pt'  # Update if needed
onnx_output = 'best.onnx'

!python export.py --weights $best_weights --img 640 --batch 1 --include onnx

# Convert ONNX to NCNN (requires ncnn and ncnn2mem tool)
# Download ncnn tools from https://github.com/Tencent/ncnn/releases or build from source
# Example command (run in shell, not Python):
# !./onnx2ncnn best.onnx best.param best.bin
# !./ncnn2mem best.param best.bin best.id.h best.id.c

print('ONNX export complete. Use ncnn tools to convert to NCNN format.')

## 5. Load Trained NCNN Model and Test on Custom Image
Use the NCNN C++/Python API to load the exported model and run inference on a custom image.

In [ ]:
# Load and test NCNN model (Python API example, if available)
# Note: Official NCNN Python API is limited; most use C++ for inference.
# Below is a placeholder for Python. For C++, use the ncnn examples from https://github.com/Tencent/ncnn

# If you have ncnn Python bindings installed:
# import ncnn
# net = ncnn.Net()
# net.load_param('best.param')
# net.load_model('best.bin')
# img = cv2.imread('/path/to/custom_image.jpg')
# ... (preprocess and run inference)

print('For NCNN inference, use the C++ API or ncnn Python bindings if available. See ncnn documentation for details.')